In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import timeit
import h5py
from pathlib import Path
import numpy as np
from itertools import product
import pickle
import scipy.special
import scipy.stats
from math import exp
import multiprocessing
import functools

from montecarlo import *

%matplotlib inline
np.seterr(all='warn')
textwidth = 6.268
mpl.rcParams['figure.dpi'] = 70
default_figargs = dict(figsize = (textwidth,textwidth))
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from montecarlo import *

To check the sanity of the monte carlo routine, the above plot compares the boltzman factor $exp(-\beta F) $ for each state of a small system (N=6) with the number of occurances seen during a monte carlo run. Each column shows these factors after an increasing number of montecarlo steps. Each row represents the states by a differnt measure, first they're simply ordered by increasing free energy, second they're plotted against free energy so that degernate states clump together. The 3rd and 4th show the overall occupation $Nf + Nc$ and the squared staggered magnetisation. 

The above plot shows the mean of each observable after x monte carlo steps, showing gradual convergence. Currently the Ff, Fc and Nc observables don't converge to the correct values!

In [102]:
#%%time
from time import time
from montecarlo import *
from general import index_histogram

class Eigenspectrum_IPR(object):
    def __init__(self, bins = 70, limit = 5):
        self.eigenval_bins = np.linspace(-limit, limit, bins + 1)
    
    def start(self, N_steps, N_sites):
        self.N_steps = N_steps
        self.eigenvals = np.zeros((N_steps,N_sites), dtype = np.float64)
        self.eigenval_histogram = np.zeros((N_steps,self.eigenval_bins.shape[0]-1), dtype = np.float64)
        self.IPR_histogram = np.zeros((N_steps,self.eigenval_bins.shape[0]-1), dtype = np.float64)

    def update(self, j, Ff, Fc, state, evals, evecs, mu, beta, J_matrix, **kwargs):
        IPRs = ((evecs * np.conj(evecs))**2).sum(axis = 0)
        self.eigenval_histogram[j], _, indices = index_histogram(self.eigenval_bins, evals)
        self.IPR_histogram[j] = np.bincount(indices, weights=IPRs, minlength = self.eigenval_bins.shape[0] + 1)[1:-1]

    
    def return_vals(self):
        E_histogram = np.mean(self.eigenval_histogram, axis = 0)
        normalisation_factor = np.sum(E_histogram)
        
        E_histogram = E_histogram / normalisation_factor 
        dE_histogram = scipy.stats.sem(self.eigenval_histogram, axis = 0) / normalisation_factor
        
        IPR_histogram, dIPR_histogram = np.mean(self.IPR_histogram, axis = 0), scipy.stats.sem(self.IPR_histogram, axis = 0)
        
        return self.eigenval_bins, E_histogram, dE_histogram, IPR_histogram, dIPR_histogram

parameters = dict(t = 1, alpha = 1.5, mu = 0, beta = 1/1.1, J = 5, U = 5, normalise = True)
n_bins = 500
MCMC_params = dict(
        N_steps = int(10 * 1000),
        N_burn_in = int(1 * 1000), 
        logger = Eigenspectrum_IPR(bins = n_bins, limit = 5),
        proposal = p_multi_site_poisson_reflect,
        proposal_args = dict(lam = 1),
        #accept_function = simple_accept,
        accept_function = perturbation_accept,
    )

#bins, E, dE, IPR, dIPR = FK_mcmc_2(parameters = parameters, **MCMC_params)

In [ ]:
def compute(N, **kwargs):
    t0 = time()
    kwargs['state'] = np.arange(N) % 2
    bins, E, dE, IPR, dIPR = FK_mcmc_2(**kwargs)
    print(f'N = {N} in t = {time() - t0:.0f} seconds')
    return bins[:-1], E, dE, IPR, dIPR

Ns = [8,16,32,64,128]
data = np.zeros(shape = (len(Ns), 5, n_bins))

with multiprocessing.Pool(multiprocessing.cpu_count() - 2) as pool:
    curried = functools.partial(compute, parameters = parameters, **MCMC_params)
    data = np.array(pool.map(curried, Ns))


np.savez('./pickled_data/IPRs.npz', 
        Ns=Ns, parameters = parameters, MCMC_params=MCMC_params, data=data, allow_pickle = True,
        )
print('done')

N = 8 in t = 17 seconds
N = 16 in t = 30 seconds
N = 32 in t = 62 seconds


/workspace/tch14/conda-envs/cmth_intelpython3_2/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: underflow encountered in square


N = 64 in t = 202 seconds


In [ ]:
from general import smooth
from itertools import count
pickled_data = np.load('./pickled_data/IPRs.npz', allow_pickle = True)
locals().update(pickled_data)
print(data.shape)

f, (Eax, Iax) = plt.subplots(2,1, figsize  = (10,10))
for i, col, N, Nslice in zip(count(), colors, Ns, data):
    bins, E, dE, IPR, dIPR = Nslice
    scale = 0.6
    E = smooth(E, scale)
    IPR = smooth(IPR, scale)
    
    Eax.plot(bins, E + 0*i/100, label = f'N = {N}')
    Iax.plot(bins, IPR + 0*i/50)
    
Eax.legend()